In [0]:
import pyspark.sql as sq
from pyspark.sql.functions import *
import os
import numpy as np

spark = sq.SparkSession.builder.master("local").appName("my app").config("spark.some.config.option", "some-value").getOrCreate()

In [0]:
poi = spark.read.csv(os.path.join(path,"POIList.csv"),header=True,inferSchema = True)
data = spark.read.csv(os.path.join(path,"DataSample.csv"),header=True,inferSchema = True)

In [0]:
data.count()

Out[55]: 22025

In [0]:
data.printSchema()

root
-- _ID: integer (nullable = true)
-- TimeSt: string (nullable = true)
-- Country: string (nullable = true)
-- Province: string (nullable = true)
-- City: string (nullable = true)
-- Latitude: double (nullable = true)
-- Longitude: double (nullable = true)

In [0]:
#***********************************
#********Data Cleanup***************
#***********************************
#Finding the suspicous IDs
suspicoius_data=(
                  data
                      .groupBy([' TimeSt', 'Latitude', 'Longitude'])
                      .agg(collect_list("_ID").alias("_ID2"))
                      .where(size("_ID2") > 1)
                ).select(explode("_ID2").alias("_ID"))
#Removing the suspicous IDs
data=data.join(aa, data._ID == aa._ID, "left_anti").drop(aa._ID)

In [0]:
data.count()

Out[58]: 17973

In [0]:
poi.show()

+-----+---------+-----------+
POIID| Latitude| Longitude|
+-----+---------+-----------+
 POI1|53.546167|-113.485734|
 POI2|53.546167|-113.485734|
 POI3|45.521629| -73.566024|
 POI4| 45.22483| -63.232729|
+-----+---------+-----------+

In [0]:
#Defining a function to calculate the distance between two point based on latitude and longitude
def dis(lat1,lon1,lat2,lon2):
    R=6371
    lon1=toRadians(lon1)
    lat1=toRadians(lat1)
    lon2=toRadians(lon2)
    lat2=toRadians(lat2)
    a=sin((lat1-lat2)/2)**2+cos(lat1)*cos(lat2)*sin((lon1-lon2)/2)**2
    c=2*atan2(sqrt(a),sqrt(1-a))
    d=R*c
    return d

In [0]:
#***************************************************
#***************Label*******************************
#***************************************************
#changing the cloumn name for poi list
poi = poi.select('POIID',col(" Latitude").alias("poiLatitude"), col("Longitude").alias("poiLongitude"))
#Cross joining the poi and data dfs
data=data.crossJoin(poi)
#Calculating the distance between two points and creating a new column
data=data.withColumn("Poidistance", dis(data['Latitude'],data['Longitude'], data['poiLatitude'],data['poiLongitude']))
data.show()

+-------+--------------------+-------+--------+---------+--------+---------+-----+-----------+------------+------------------+
 _ID| TimeSt|Country|Province| City|Latitude|Longitude|POIID|poiLatitude|poiLongitude| Poidistance|
+-------+--------------------+-------+--------+---------+--------+---------+-----+-----------+------------+------------------+
4516516|2017-06-21 00:00:...| CA| ON| Waterloo|43.49347|-80.49123| POI1| 53.546167| -113.485734|2643.7525680899066|
4516516|2017-06-21 00:00:...| CA| ON| Waterloo|43.49347|-80.49123| POI2| 53.546167| -113.485734|2643.7525680899066|
4516516|2017-06-21 00:00:...| CA| ON| Waterloo|43.49347|-80.49123| POI3| 45.521629| -73.566024| 593.4134412324524|
4516516|2017-06-21 00:00:...| CA| ON| Waterloo|43.49347|-80.49123| POI4| 45.22483| -63.232729| 1382.79036317354|
4516547|2017-06-21 18:00:...| CA| ON| London| 42.9399| -81.2709| POI1| 53.546167| -113.485734|2630.7162097829278|
4516547|2017-06-21 18:00:...| CA| ON| London| 42.9399| -81.2709| POI2| 53.546167| -113.485734|2630.7162097829278|
4516547|2017-06-21 18:00:...| CA| ON| London| 42.9399| -81.2709| POI3| 45.521629| -73.566024| 677.3096549572066|
4516547|2017-06-21 18:00:...| CA| ON| London| 42.9399| -81.2709| POI4| 45.22483| -63.232729|1459.8356696016303|
4516550|2017-06-21 15:00:...| CA| ON| Guelph| 43.5776| -80.2201| POI1| 53.546167| -113.485734| 2655.754228795986|
4516550|2017-06-21 15:00:...| CA| ON| Guelph| 43.5776| -80.2201| POI2| 53.546167| -113.485734| 2655.754228795986|
4516550|2017-06-21 15:00:...| CA| ON| Guelph| 43.5776| -80.2201| POI3| 45.521629| -73.566024| 569.6477373989128|
4516550|2017-06-21 15:00:...| CA| ON| Guelph| 43.5776| -80.2201| POI4| 45.22483| -63.232729| 1359.333110507519|
4516600|2017-06-21 15:00:...| CA| ON|Stratford| 43.3716| -80.9773| POI1| 53.546167| -113.485734| 2620.447164403852|
4516600|2017-06-21 15:00:...| CA| ON|Stratford| 43.3716| -80.9773| POI2| 53.546167| -113.485734| 2620.447164403852|
4516600|2017-06-21 15:00:...| CA| ON|Stratford| 43.3716| -80.9773| POI3| 45.521629| -73.566024| 634.7339960242033|
4516600|2017-06-21 15:00:...| CA| ON|Stratford| 43.3716| -80.9773| POI4| 45.22483| -63.232729| 1424.165830534854|
4516613|2017-06-21 15:00:...| CA| ON|Stratford| 43.3716| -80.9773| POI1| 53.546167| -113.485734| 2620.447164403852|
4516613|2017-06-21 15:00:...| CA| ON|Stratford| 43.3716| -80.9773| POI2| 53.546167| -113.485734| 2620.447164403852|
4516613|2017-06-21 15:00:...| CA| ON|Stratford| 43.3716| -80.9773| POI3| 45.521629| -73.566024| 634.7339960242033|
4516613|2017-06-21 15:00:...| CA| ON|Stratford| 43.3716| -80.9773| POI4| 45.22483| -63.232729| 1424.165830534854|
+-------+--------------------+-------+--------+---------+--------+---------+-----+-----------+------------+------------------+
only showing top 20 rows

In [0]:
#Assign each request to the closest poi
data_with_min_dis=data.groupBy('_ID').min('Poidistance')
data=data.join(data_with_min_dis,(data['_ID'] == data_with_min_dis['_ID']) & (data['Poidistance'] == data_with_min_dis['min(Poidistance)'])).drop(data_with_min_dis._ID)

data=data.drop('Poidistance')


In [0]:
data.show()

+--------------------+-------+--------+------------+--------+---------+-----+-----------+------------+-------+-----------------+
 TimeSt|Country|Province| City|Latitude|Longitude|POIID|poiLatitude|poiLongitude| _ID| min(Poidistance)|
+--------------------+-------+--------+------------+--------+---------+-----+-----------+------------+-------+-----------------+
2017-06-21 00:00:...| CA| ON| Waterloo|43.49347|-80.49123| POI3| 45.521629| -73.566024|4516516|593.4134412324524|
2017-06-21 18:00:...| CA| ON| London| 42.9399| -81.2709| POI3| 45.521629| -73.566024|4516547|677.3096549572066|
2017-06-21 15:00:...| CA| ON| Guelph| 43.5776| -80.2201| POI3| 45.521629| -73.566024|4516550|569.6477373989128|
2017-06-21 15:00:...| CA| ON| Stratford| 43.3716| -80.9773| POI3| 45.521629| -73.566024|4516600|634.7339960242033|
2017-06-21 15:00:...| CA| ON| Stratford| 43.3716| -80.9773| POI3| 45.521629| -73.566024|4516613|634.7339960242033|
2017-06-21 14:00:...| CA| ON| Kitchener| 43.4381| -80.5099| POI3| 45.521629| -73.566024|4516693|597.3787433319695|
2017-06-21 10:00:...| CA| ON| Sarnia| 42.961| -82.373| POI3| 45.521629| -73.566024|4516771|756.6282143405344|
2017-06-21 15:00:...| CA| ON| London| 43.0091| -81.1765| POI3| 45.521629| -73.566024|4516915| 666.9416656644|
2017-06-21 16:00:...| CA| ON| Kitchener| 43.4278| -80.5135| POI3| 45.521629| -73.566024|4516953|598.1304004974271|
2017-06-21 01:00:...| CA| ON| Kitchener| 43.4381| -80.5099| POI3| 45.521629| -73.566024|4516966|597.3787433319695|
2017-06-21 21:00:...| CA| ON| Windsor| 42.2825| -83.0372| POI3| 45.521629| -73.566024|4517044|839.2393067655215|
2017-06-21 22:00:...| CA| ON| Windsor|42.24397|-82.98058| POI3| 45.521629| -73.566024|4517081|837.2215116755591|
2017-06-21 19:00:...| CA| ON|Saint Thomas| 42.7792| -81.1927| POI3| 45.521629| -73.566024|4517175|680.2278980833039|
2017-06-21 18:00:...| CA| ON| Waterloo| 43.4634| -80.5201| POI3| 45.521629| -73.566024|4517220|596.9319139426241|
2017-06-21 03:00:...| CA| ON| Chatham| 42.4247| -82.1755| POI3| 45.521629| -73.566024|4517250|769.6611559032159|
2017-06-21 04:00:...| CA| ON| Guelph| 43.5425| -80.2676| POI3| 45.521629| -73.566024|4517326|574.7565039488508|
2017-06-21 14:00:...| CA| ON| Chatham| 42.4247| -82.1755| POI3| 45.521629| -73.566024|4517349|769.6611559032159|
2017-06-21 08:00:...| CA| ON| Guelph| 43.5215| -80.2338| POI3| 45.521629| -73.566024|4517420| 573.272810303738|
2017-06-21 03:00:...| CA| ON| Waterloo| 43.4678| -80.584| POI3| 45.521629| -73.566024|4517466|601.4060727323244|
2017-06-21 21:00:...| CA| ON| Waterloo| 43.4634| -80.5201| POI3| 45.521629| -73.566024|4517518|596.9319139426241|
+--------------------+-------+--------+------------+--------+---------+-----+-----------+------------+-------+-----------------+
only showing top 20 rows

In [0]:
#************************************************
#*****************Analysis***********************
#************************************************
#calculate the average and standard deviation of the distance between the POI to each of its assigned requests.
analysis1=data.groupBy('POIID').agg(avg("min(Poidistance)").alias('Average'), stddev("min(Poidistance)").alias('Standard_Deviation'))
analysis1.show()

+-----+------------------+------------------+
POIID| Average|Standard_Deviation|
+-----+------------------+------------------+
 POI4| 514.9971719812205| 1506.889970770323|
 POI2| 300.7147475686817| 388.2733852635428|
 POI1| 300.7147475686817| 388.2733852635428|
 POI3|451.65114920151524|223.63174183102885|
+-----+------------------+------------------+

In [0]:
#Calculate the radius and density (requests/area) for each POI.
analysis2=data.groupBy('POIID').agg(max("min(Poidistance)").alias('Radius'), count("min(Poidistance)").alias('Count'))
analysis2=analysis2.withColumn('density',analysis2['Count']/(analysis2['Radius']**2*np.pi))
analysis2.show()

+-----+------------------+-----+--------------------+
POIID| Radius|Count| density|
+-----+------------------+-----+--------------------+
 POI4| 9349.572770487366| 422|1.536664455904176...|
 POI2| 11531.82083183645| 8749|2.094174038984838...|
 POI1| 11531.82083183645| 8749|2.094174038984838...|
 POI3|1474.5809620285695| 8802|0.001288529145748...|
+-----+------------------+-----+--------------------+

In [0]:
#***************************************************
#*****************Model*****************************
#***************************************************
#From analysis 2, we have calculated the density and radius of each POI.
#To visualize the popularity of each POI, density has to be mapped between -10 and 10.
#Mathematical formula for that is f(x)=(b-a)*(x-min)/(max-min)+a; b=10, a=-10

analysis3=analysis2.toPandas() 
minn=np.min(analysis3['density'])
maxx=np.max(analysis3['density'])
analysis3['density(mapped)']=20*(analysis3['density']-minn)/(maxx-minn)-10
analysis3

Out[66]:

,POIID,Radius,Count,density,density(mapped)
0,POI4,9349.572770,422,0.000002,-10.000000
1,POI2,11531.820832,8749,0.000021,-9.698443
2,POI1,11531.820832,8749,0.000021,-9.698443
3,POI3,1474.580962,8802,0.001289,10.000000
